In [1]:
%env DATASET_NAME=AMRB2_species
%env MANIFOLD_D=512
%env MODEL_NAME=resnet
%env TRAIN_EPOCHS=50

env: DATASET_NAME=AMRB2_species
env: MANIFOLD_D=512
env: MODEL_NAME=resnet
env: TRAIN_EPOCHS=50


In [2]:
import logging
import os
from typing import Literal

import numpy as np
import torch
import sklearn.metrics
from tqdm import tqdm

from config import Config, load_config
from datasets import init_dataloaders, init_labels, init_shape
from models import get_model_optimizer_and_step
from models.common import gen_topk_accs, load_model_state, save_model_state
from matplotlib import pyplot as plt
from PIL import Image

ModuleNotFoundError: No module named 'torch'

In [3]:
# initialize the RNG deterministically
np.random.seed(42)
torch.manual_seed(42)

config = load_config()

# initialize data attributes and loaders
init_labels(config)
init_shape(config)
init_dataloaders(config)

config.print_labels()

import wandb.plot

import wandb

wandb.init(
    project="uq_ood",
    name=config.run_name,
    config=config.run_config,
)

INFO:root:LOG_LEVEL=INFO
INFO:root:OOD_K=
INFO:root:DATA_DIR=/home/pjaya001/datasets
INFO:root:DATASET_NAME=AMRB2_species
INFO:root:MODEL_NAME=resnet
INFO:root:EXPERIMENT_BASE=/home/pjaya001/experiments/ood_flows
INFO:root:MANIFOLD_D=512
INFO:root:BATCH_SIZE=32
INFO:root:OPTIM_LR=0.001
INFO:root:OPTIM_M=0.8
INFO:root:TRAIN_EPOCHS=50
INFO:root:EXC_RESUME=1
INFO:root:Using device: cuda
INFO:root:Dataset file: /home/pjaya001/datasets/AMRB2/ctr_1_fit_f32.imag.npz
INFO:root:[preparation] loaded target info
INFO:root:[preparation] performed train/test split
INFO:root:Prepared datasets in 17.589163303375244 s


Performing ind/ood split


INFO:root:Labels (train, test): ['Acinetobacter', 'B_subtilis', 'E_coli', 'K_pneumoniae', 'S_aureus']
INFO:root:Labels (ood): []


Performed ind/ood split
323968 215968 0


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yasith. Use `wandb login --relogin` to force relogin


In [5]:
def epoch_stats_to_wandb(
    stats: dict[str, dict],
    config: Config,
    step: int,
) -> dict:
    # initialize metric dict
    metrics = {}
    metrics["trn/loss"] = None
    metrics["trn/acc"] = dict(acc1=None, acc2=None, acc3=None)
    metrics["tid/loss"] = None
    metrics["tid/acc"] = dict(acc1=None, acc2=None, acc3=None)
    metrics["tod/loss"] = None

    # get label information
    ind_labels, ood_labels = config.get_ind_labels(), config.get_ood_labels()
    Ki, Ko = len(ind_labels), len(ood_labels)

    figs = {}
    if "trn" in stats:
        metrics["trn/loss"] = stats["trn"]["loss"]
        y_true_trn: np.ndarray = stats["trn"]["y_true"]
        y_prob_trn: np.ndarray = stats["trn"]["y_prob"]
        metrics["trn/acc"] = gen_topk_accs(y_true_trn, y_prob_trn, Ki)

        cm = sklearn.metrics.confusion_matrix(
            y_true_trn, y_prob_trn.argmax(-1), labels=list(range(Ki))
        )
        disp = sklearn.metrics.ConfusionMatrixDisplay(cm, display_labels=ind_labels)
        disp.plot()
        fp = os.path.join(config.experiment_path, f"cm_trn_e{step}.png")
        plt.savefig(fp)
        figs["trn/cm"] = wandb.Image(Image.open(fp))

    y_true_tst = []
    y_prob_tst = []

    if "tid" in stats:
        y_true_tid: np.ndarray = stats["tid"]["y_true"]
        y_prob_tid: np.ndarray = stats["tid"]["y_prob"]
        if len(y_true_tid) > 0:
            metrics["tid/loss"] = stats["tid"]["loss"]
            metrics["tid/acc"] = gen_topk_accs(y_true_tid, y_prob_tid, Ki)
            y_true_tst.append(y_true_tid)
            y_prob_tst.append(y_prob_tid)

    if "tod" in stats:
        y_true_tod: np.ndarray = stats["tod"]["y_true"]
        y_prob_tod: np.ndarray = stats["tod"]["y_prob"]
        if len(y_true_tod) > 0:
            metrics["tod/loss"] = stats["tod"]["loss"]
            y_true_tst.append(y_true_tod)
            y_prob_tst.append(y_prob_tod)

    # concatenate y of tid and tod datasets
    y_true_tst = np.concatenate(y_true_tst, axis=0)
    y_prob_tst = np.concatenate(y_prob_tst, axis=0)

    # zero-pad y_prob_tst for ood targets
    y_prob_tst = np.pad(y_prob_tst, pad_width=((0, 0), (0, Ko)))
    perm_labels = ind_labels + ood_labels

    cm = sklearn.metrics.confusion_matrix(
        y_true_tst, y_prob_tst.argmax(-1), labels=list(range(Ki + Ko))
    )
    disp = sklearn.metrics.ConfusionMatrixDisplay(cm, display_labels=perm_labels)
    disp.plot()
    fp = os.path.join(config.experiment_path, f"cm_tst_e{step}.png")
    plt.savefig(fp)
    figs["tst/cm"] = wandb.Image(Image.open(fp))

    tqdm.write(f"Epoch {step}: {metrics}")

    join = lambda x: ",".join(map(perm_labels.__getitem__, x))
    fig_row_lbl = []
    fig_a_data = []
    fig_a_col_lbl = []
    fig_b_data = []
    fig_b_col_lbl = []

    if "trn" in stats:
        fig_row_lbl.append("trn")
        (E_x_trn, E_y_trn, E_xp_trn, E_yp_trn) = zip(*stats["trn"]["samples"])
        E_x_trn = np.concatenate(E_x_trn, axis=1)
        E_xp_trn = np.concatenate(E_xp_trn, axis=1)
        fig_a_data.append(E_x_trn)
        fig_a_col_lbl.append(join(E_y_trn))
        fig_b_data.append(E_xp_trn)
        fig_b_col_lbl.append(join(E_yp_trn))

    if "tid" in stats and len(stats["tid"]["y_true"]) > 0:
        fig_row_lbl.append("tid")
        (E_x_tid, E_y_tid, E_xp_tid, E_yp_tid) = zip(*stats["tid"]["samples"])
        E_x_tid = np.concatenate(E_x_tid, axis=1)
        E_xp_tid = np.concatenate(E_xp_tid, axis=1)
        fig_a_data.append(E_x_tid)
        fig_a_col_lbl.append(join(E_y_tid))
        fig_b_data.append(E_xp_tid)
        fig_b_col_lbl.append(join(E_yp_tid))

    if "tod" in stats and len(stats["tod"]["y_true"]) > 0:
        fig_row_lbl.append("tod")
        (E_x_tod, E_y_tod, E_xp_tod, E_yp_tod) = zip(*stats["tod"]["samples"])
        E_x_tod = np.concatenate(E_x_tod, axis=1)
        E_xp_tod = np.concatenate(E_xp_tod, axis=1)
        fig_a_data.append(E_x_tod)
        fig_a_col_lbl.append(join(E_y_tod))
        fig_b_data.append(E_xp_tod)
        fig_b_col_lbl.append(join(E_yp_tod))

    fig_a_data = np.concatenate(fig_a_data, axis=0)
    fig_b_data = np.concatenate(fig_b_data, axis=0)

    fig_row_lbl = ", ".join([f"R{i+1}={v}" for i, v in enumerate(fig_row_lbl)])
    fig_a_col_lbl = ", ".join([f"R{i+1}={v}" for i, v in enumerate(fig_a_col_lbl)])
    fig_b_col_lbl = ", ".join([f"R{i+1}={v}" for i, v in enumerate(fig_b_col_lbl)])
    fig_a_cap = f"sample inputs - rows: [{fig_row_lbl}] - targets: [{fig_a_col_lbl}]"
    fig_b_cap = f"sample output - rows: [{fig_row_lbl}] - targets: [{fig_b_col_lbl}]"

    figs["samples/input"] = wandb.Image(fig_a_data, caption=fig_a_cap)
    figs["samples/output"] = wandb.Image(fig_b_data, caption=fig_b_cap)

    done_keys = ["y_true", "y_prob", "y_ucty", "samples"]

    for prefix in ["trn", "tid", "tod"]:
        if prefix not in stats:
            continue
        prefix_stats: dict = stats[prefix]
        for key in set(prefix_stats).difference(done_keys):
            val = prefix_stats[key]
            # unbounded histograms
            if key in ["u_norm", "v_norm", "z_norm", "z_nll"]:
                val = np.tanh(val)
                # save v_norm for AUROC computation
                if key in ["v_norm"]:
                    figs[f"{prefix}/{key}"] = val
                hist = np.histogram(val, bins=100, range=(0.0, 1.0))
                figs[f"{prefix}/{key}_hist"] = wandb.Histogram(np_histogram=hist)
            # bounded histograms
            elif key == "y_ucty":
                hist = np.histogram(val, bins=100, range=(0.0, 1.0))
                figs[f"{prefix}/{key}_hist"] = wandb.Histogram(np_histogram=hist)
            # log everything else
            else:
                figs[f"{prefix}/{key}"] = val

    prefix = "ood_detection"
    if "v_norm" in stats["tid"] and "v_norm" in stats["tod"]:
        tid_v_norm = stats["tid"]["v_norm"]
        tod_v_norm = stats["tod"]["v_norm"]
        B_InD = tid_v_norm.shape[0]
        B_OoD = tod_v_norm.shape[0]
        # binary classification labels for ID and OOD
        LABELS = ["InD", "OoD"]
        values = np.concatenate([tid_v_norm, tod_v_norm], axis=0)
        values_2d = np.stack([1.0 - values, values], axis=-1)
        target = np.concatenate([np.zeros(B_InD), np.ones(B_OoD)], axis=0)
        figs[f"{prefix}/roc"] = wandb.plot.roc_curve(target, values_2d, LABELS)
        figs[f"{prefix}/pr"] = wandb.plot.pr_curve(target, values_2d, LABELS)
        figs[f"{prefix}/auroc"] = sklearn.metrics.roc_auc_score(target, values)

    data = {}
    data.update(metrics)
    data.update(figs)
    return data

In [6]:
assert config.train_loader
assert config.test_loader

model, optim, step = get_model_optimizer_and_step(config)

# load saved model and optimizer, if present
load_model_state(model, config)
model = model.float().to(config.device)

wandb.watch(model, log_freq=100)

# run train / test loops
logging.info("Started Train/Test")

artifact = wandb.Artifact(f"{config.run_name}-{config.model_name}", type="model")

# loop over epochs
for epoch in range(1, config.train_epochs + 1):
    epoch_stats: dict = {}

    # train
    trn_stats = step(
        prefix="train",
        model=model,
        epoch=epoch,
        config=config,
        optim=optim,
    )
    epoch_stats["trn"] = trn_stats

    # test (InD)
    tid_stats = step(
        prefix="test_ind",
        model=model,
        epoch=epoch,
        config=config,
    )
    epoch_stats["tid"] = tid_stats

    # test (OoD)
    tod_stats = step(
        prefix="test_ood",
        model=model,
        epoch=epoch,
        config=config,
    )
    epoch_stats["tod"] = tod_stats

    wandb.log(epoch_stats_to_wandb(epoch_stats, config, epoch), step=epoch)

    # save model and optimizer states
    save_model_state(model, config, epoch)
    fp = config.experiment_path
    model_name = config.model_name
    artifact.add_file(os.path.join(fp, f"{model_name}_model_e{epoch}.pth"))

artifact.save()

INFO:root:Started Train/Test


[train] Epoch 1: 100%|██████████| 10124/10124, Loss(agg)=0.6204, Loss(y)=0.6181, Loss(x)=0.0023
[test_ind] Epoch 1: 100%|██████████| 6749/6749, Loss(agg)=0.5445, Loss(y)=0.5422, Loss(x)=0.0023
[test_ood] Epoch 1: |          | 0/0
Epoch 1: {'trn/loss': {'agg': 0.5612607961090033, 'y': 0.5453838400339619, 'x': 0.015876956454431795}, 'trn/acc': {'acc1': 0.6273026965626235, 'acc2': 0.8643230195574871, 'acc3': 0.9442691870802055}, 'tid/loss': {'agg': 0.66398042114969, 'y': 0.6616743356894698, 'x': 0.002306085472225763}, 'tid/acc': {'acc1': 0.5404596977329975, 'acc2': 0.8051655800859386, 'acc3': 0.9140150392650763}, 'tod/loss': None}


INFO:root:saving model state - e1


[train] Epoch 2: 100%|██████████| 10124/10124, Loss(agg)=0.5143, Loss(y)=0.5138, Loss(x)=0.0005
[test_ind] Epoch 2: 100%|██████████| 6749/6749, Loss(agg)=0.4925, Loss(y)=0.4921, Loss(x)=0.0004
[test_ood] Epoch 2: |          | 0/0


INFO:root:saving model state - e2


...

Epoch 40: {'trn/loss': {'agg': 0.6268999565941739, 'y': 0.6266068503796371, 'x': 0.00029310587502602076}, 'trn/acc': {'acc1': 0.6453353417621494, 'acc2': 0.7223799881469775, 'acc3': 0.8232263680363493}, 'tid/loss': {'agg': 0.9740812670903695, 'y': 0.9738490988429626, 'x': 0.00023216823228789586}, 'tid/acc': {'acc1': 0.48215013335308937, 'acc2': 0.5949307304785895, 'acc3': 0.8067398873907246}, 'tod/loss': None}
[train] Epoch 41: 100%|██████████| 10124/10124, Loss(agg)=0.6669, Loss(y)=0.6666, Loss(x)=0.0003
[test_ind] Epoch 41: 100%|██████████| 6749/6749, Loss(agg)=1.0851, Loss(y)=1.0845, Loss(x)=0.0006
[test_ood] Epoch 41: |          | 0/0


INFO:root:saving model state - e41


Epoch 41: {'trn/loss': {'agg': 0.6263543794933207, 'y': 0.6260470721512997, 'x': 0.00030730741230255054}, 'trn/acc': {'acc1': 0.6485949229553536, 'acc2': 0.7228831242591861, 'acc3': 0.8231368530225207}, 'tid/loss': {'agg': 0.998624093321663, 'y': 0.9983269012862797, 'x': 0.0002971919073248143}, 'tid/acc': {'acc1': 0.4792654467328493, 'acc2': 0.6057008445695659, 'acc3': 0.8049201733590161}, 'tod/loss': None}
[train] Epoch 42: 100%|██████████| 10124/10124, Loss(agg)=0.7431, Loss(y)=0.7428, Loss(x)=0.0003
[test_ind] Epoch 42: 100%|██████████| 6749/6749, Loss(agg)=1.4335, Loss(y)=1.4332, Loss(x)=0.0002
[test_ood] Epoch 42: |          | 0/0


INFO:root:saving model state - e42


Epoch 42: {'trn/loss': {'agg': 0.6241018201100501, 'y': 0.6238099648394588, 'x': 0.0002918557169817776}, 'trn/acc': {'acc1': 0.6475454365863295, 'acc2': 0.7218737653101541, 'acc3': 0.8233930511655472}, 'tid/loss': {'agg': 0.9494973367665464, 'y': 0.9492403025591103, 'x': 0.0002570338665559758}, 'tid/acc': {'acc1': 0.46404559934805156, 'acc2': 0.6046543932434435, 'acc3': 0.818135094088013}, 'tod/loss': None}
[train] Epoch 43: 100%|██████████| 10124/10124, Loss(agg)=0.9364, Loss(y)=0.9362, Loss(x)=0.0002
[test_ind] Epoch 43:  53%|█████▎    | 3551/6749, Loss(agg)=0.8258, Loss(y)=0.8255, Loss(x)=0.0003